# Modelling of network components
https://docs.netsquid.org/latest-release/tutorial.components.html

## Channels

In [3]:
import netsquid as ns
from netsquid.components import Channel, QuantumChannel
channel = Channel(name='MyChannel')

In [4]:
channel.send('hello world')
ns.sim_run()

SimStats()

In [5]:
items, delay = channel.receive()
print(items)
print(delay)

['hello world']
0.0


In [6]:
from netsquid.components.models.delaymodels import FixedDelayModel
fixed_delaymodel = FixedDelayModel(delay=10)

In [8]:
channel.models['delay_model'] = fixed_delaymodel
channel.send('hello world')
ns.sim_run()
channel.receive()

(['hello world'], 10.0)

In [9]:
from netsquid.components.models.delaymodels import GaussianDelayModel
gaussian_delaymodel = GaussianDelayModel(delay_mean=5, delay_std=0.1)

In [10]:
Channel('tutorialChannel', length=10)

Channel(name='tutorialChannel')

In [11]:
from netsquid.components.models.delaymodels import FibreDelayModel
delay_model = FibreDelayModel()
print(f"speed of  light in fibre: {delay_model.properties['c']:.1f} [km/s]")
delay_model.required_properties

speed of  light in fibre: 200000.0 [km/s]


['length']

In [13]:
from netsquid.components.models.qerrormodels import FibreLossModel
from netsquid.components.qchannel import QuantumChannel
loss_model = FibreLossModel(p_loss_init=0.83, p_loss_length=0.2)
qchannel = QuantumChannel('MyQChannel',  length=20, models={'quantum_loss_model': loss_model})

## Quantum memory

In [15]:
from netsquid.components import QuantumMemory
qmem = QuantumMemory(name='MyMemory', num_positions=1)

In [16]:
from netsquid.components.models.qerrormodels import DepolarNoiseModel
depolar_noise = DepolarNoiseModel(depolar_rate=1e6)

In [17]:
qmem = QuantumMemory('DepolarMemory', num_positions=2, memory_noise_models=[depolar_noise, depolar_noise])

In [18]:
from netsquid.qubits.qubitapi import create_qubits
qubits = create_qubits(1)
qmem.put(qubits)
qmem.peek(0)

[Qubit('QS#0-0')]

In [19]:
qmem.pop(positions=0)

[Qubit('QS#0-0')]

In [20]:
qmem.peek(0)

[None]

In [51]:
import netsquid.qubits.operators as ops
qmem.put(qubits)
qmem.operate(ops.X, positions=[0])

In [54]:
qmem.measure(positions=[0])

([0], [1.0])

In [55]:
qmem.measure(positions=[0], observable=ops.X)

([1], [0.4999999999999998])

## Ports

In [56]:
channel = Channel('tutorialChannel', delay=3)
channel.ports['send'].tx_input('hello')
ns.sim_run()
channel.ports['recv'].rx_output()

Message(items=['hello'], header=None, ch_delta_time=3.0)

In [57]:
channel.ports['recv'].connect(qmem.ports['qin0'])

In [58]:
qubit, = create_qubits(1)
print(qubit)

Qubit('QS#1-0')


In [59]:
channel.send(qubit)
ns.sim_run()

SimStats()

In [60]:
qmem.peek(0)

[Qubit('QS#1-0')]

## Ping pong using compoments and ports

In [62]:
from netsquid.components.component import Port
import pydynaa

In [69]:
class PingEntity(pydynaa.Entity):
    length = 2e-3   # channel length (km)

    def __init__(self):
        # create a memory and a quantum channel
        self.qmemory = QuantumMemory('PingMemory', num_positions=1)
        self.qchannel = QuantumChannel('PingChannel', length=self.length, models={'delay_model': FibreDelayModel()})
        # link output from qmemory (pop) to input of ping channel
        self.qmemory.ports['qout'].connect(self.qchannel.ports['send'])
        # setup callback function to handle input on quantum memory port 'qin0'
        self._wait(pydynaa.EventHandler(self._handle_input_qubit), entity=self.qmemory.ports['qin0'], event_type=Port.evtype_input)
        self.qmemory.ports['qin0'].notify_all_input = True

    def start(self, qubit):
        # start the game by having ping player send the first qubit (ping)
        self.qchannel.send(qubit)

    def wait_for_pong(self, other_entity):
        # setup this entity to pass incoming qubits to its quantum memory
        self.qmemory.ports['qin0'].connect(other_entity.qchannel.ports['recv'])

    def _handle_input_qubit(self, event):
        # callback function challed by the pong handler when pong event is triggered
        [m], [prob] = self.qmemory.measure(positions=[0], observable=ns.Z)
        labels_z = ('|0>', '|1>')
        print(f'{ns.sim_time():.1f}: Pong event! PingEntity measured {labels_z[m]} with probability {prob:.2f}')
        self.qmemory.pop(positions=[0])


class PongEntity(pydynaa.Entity):
    length = 2e-3

    def __init__(self):
        # create a memory and a quantum channel
        self.qmemory = QuantumMemory('PongMemory', num_positions=1)
        self.qchannel = QuantumChannel('PingChannel', length=self.length, models={'delay_model': FibreDelayModel()})
        # link output from qmemory (pop) to input of ping channel
        self.qmemory.ports['qout'].connect(self.qchannel.ports['send'])
        # setup callback function to handle input on quantum memory:
        self._wait(pydynaa.EventHandler(self._handle_input_qubit), entity=self.qmemory.ports['qin0'], event_type=Port.evtype_input)
        self.qmemory.ports['qin0'].notify_all_input = True

    def wait_for_ping(self, other_entity):
        # setup this entity to pass incoming qubits to its quantum memory
        self.qmemory.ports['qin0'].connect(other_entity.qchannel.ports['recv'])

    def _handle_input_qubit(self, event):
        # callback function called by the pong handler when pong event is triggered
        [m], [prob] = self.qmemory.measure(positions=[0], observable=ns.X)
        labels_x = ('|+>', '|->')
        print(f'{ns.sim_time():.1f}: Ping event! PongEntity measured {labels_x[m]} with probability {prob:.2f}')
        self.qmemory.pop(positions=[0])


In [70]:
# create entities and register them to each other
ns.sim_reset()
ping = PingEntity()
pong = PongEntity()
ping.wait_for_pong(pong)
pong.wait_for_ping(ping)
# create a qubit and instruct the ping entity to start
qubit, = ns.qubits.create_qubits(1)
ping.start(qubit)

In [71]:
ns.set_random_state(seed=42)
stats = ns.sim_run(91)

10.0: Ping event! PongEntity measured |+> with probability 0.50
20.0: Pong event! PingEntity measured |1> with probability 0.50
30.0: Ping event! PongEntity measured |-> with probability 0.50
40.0: Pong event! PingEntity measured |1> with probability 0.50
50.0: Ping event! PongEntity measured |+> with probability 0.50
60.0: Pong event! PingEntity measured |0> with probability 0.50
70.0: Ping event! PongEntity measured |+> with probability 0.50
80.0: Pong event! PingEntity measured |1> with probability 0.50
90.0: Ping event! PongEntity measured |-> with probability 0.50


## Quantum teleportation using components

In [203]:
ns.set_qstate_formalism(ns.QFormalism.DM)
ns.sim_reset()

In [204]:
class Alice(pydynaa.Entity):
    def __init__(self, teleport_state, cchannel_send_port):
        self.teleport_state = teleport_state
        self.cchannel_send_port = cchannel_send_port
        self.qmemory = QuantumMemory('AliceMemory', num_positions=2)
        self._wait(pydynaa.EventHandler(self._handle_input_qubit), entity=self.qmemory.ports['qin1'], event_type=Port.evtype_input)
        self.qmemory.ports['qin1'].notify_all_input = True

    def _handle_input_qubit(self, event):
        # callback function that does teleportation and schedules a corrections ready event
        q0, = ns.qubits.create_qubits(1, no_state=True)
        ns.qubits.assign_qstate([q0], self.teleport_state)
        self.qmemory.put([q0], positions=[0])
        self.qmemory.operate(ns.CNOT, positions=[0, 1])
        self.qmemory.operate(ns.H, positions=[0])
        m0, m1 = self.qmemory.measure(positions=[0, 1], observable=ns.Z, discard=True)[0]
        self.cchannel_send_port.tx_input([m0, m1])
        print(f'{ns.sim_time():.1f}: Alice received entangled qubit, measured qubits & sending corrections')

class Bob(pydynaa.Entity):
    depolar_rate = 1e7   # depolarization rate of waiting qubits [Hz]
    
    def __init__(self, cchannel_recv_port):
        noise_model = DepolarNoiseModel(depolar_rate = self.depolar_rate)
        self.qmemory = QuantumMemory('BobMemory', num_positions=1, memory_noise_models=[noise_model])
        cchannel_recv_port.bind_output_handler(self._handle_corrections)
    
    def _handle_corrections(self, message):
        # callback function that handles messages from both alice and charlie
        m0, m1 = message.items
        if m1:
            self.qmemory.operate(ns.X, positions=[0])
        if m0:
            self.qmemory.operate(ns.Z, positions=[0])
        qubit = self.qmemory.pop(positions=[0])
        fidelity = ns.qubits.fidelity(qubit, ns.y0, squared=True)
        print(f'{ns.sim_time():.1f} bob received entangled qubit and corrections! Fidelity = {fidelity:.3f}')

In [205]:
from netsquid.qubits.state_sampler import StateSampler
import netsquid.qubits.ketstates as ks
state_sampler = StateSampler([ks.b00], [1.0])

from netsquid.components.qsource import QSource, SourceStatus
charlie_source = QSource('Charlie', state_sampler, frequency=100, num_ports=2, \
                         timing_model=FixedDelayModel(delay=50), status=SourceStatus.INTERNAL)

In [206]:
def setup_network(alice, bob, qsource, length=4e-3):
    qchannel_c2a = QuantumChannel('Charlie->Alice', length=length/2, models={'delay_model':FibreDelayModel()})
    qchannel_c2b = QuantumChannel('Charlie->Bob', length=length/2, models={'delay_model': FibreDelayModel()})
    qsource.ports['qout0'].connect(qchannel_c2a.ports['send'])
    qsource.ports['qout1'].connect(qchannel_c2b.ports['send'])
    alice.qmemory.ports['qin1'].connect(qchannel_c2a.ports['recv'])
    bob.qmemory.ports['qin0'].connect(qchannel_c2b.ports['recv'])

In [207]:
from netsquid.components import ClassicalChannel
cchannel = ClassicalChannel('CChannel', length=4e-3, models={'delay_model': FibreDelayModel()})
alice = Alice(teleport_state=ns.y0, cchannel_send_port=cchannel.ports['send'])
bob = Bob(cchannel_recv_port=cchannel.ports['recv'])
setup_network(alice, bob, charlie_source)

stats = ns.sim_run(end_time=100)

10.0: Alice received entangled qubit, measured qubits & sending corrections
30.0 bob received entangled qubit and corrections! Fidelity = 0.909
60.0: Alice received entangled qubit, measured qubits & sending corrections
80.0 bob received entangled qubit and corrections! Fidelity = 0.909
